# test the fracture function

In [8]:
from ultralytics import YOLO
import os
import json
import supervision as sv
import cv2
import numpy as np
from shapely.geometry import Polygon

# 加載訓練好的 YOLO 模型
model = YOLO('./fracture_detection_model/best.pt')

# 路徑設置
images_folder = "./temp/data/images"
small_output_folder = "./temp/output_1/cropped"
annotations_folder = "./temp/data/annotations"
completed_output_folder = "./temp/output_2/completed_output"
origin_txt_file = "./temp/output_1/origin.txt"

# 創建輸出文件夾
os.makedirs(completed_output_folder, exist_ok=True)

# 讀取 origin.txt 文件，存儲偏移值
offsets = {}
with open(origin_txt_file, "r") as f:
    for line in f:
        parts = line.strip().split(":")
        img_file = parts[0].strip()
        coords = parts[1].strip()
        coords_dict = {kv.split("=")[0].strip(): int(kv.split("=")[1].strip()) for kv in coords.split(", ")}
        offsets[img_file] = (coords_dict["x1"], coords_dict["y1"])

mean_iou_list = []

# 定義 IoU 計算函數（基於四邊形）
def calculate_polygon_iou(boxA, boxB):
    """
    計算兩個四邊形的 IoU
    :param boxA: 第一個四邊形的頂點座標 [(x1, y1), (x2, y2), (x3, y3), (x4, y4)]
    :param boxB: 第二個四邊形的頂點座標 [(x1, y1), (x2, y2), (x3, y3), (x4, y4)]
    :return: IoU 值
    """
    polyA = Polygon(boxA)
    polyB = Polygon(boxB)

    if not polyA.is_valid or not polyB.is_valid:
        return 0

    intersection_area = polyA.intersection(polyB).area
    union_area = polyA.union(polyB).area

    return intersection_area / union_area if union_area > 0 else 0

# 處理每張圖片
for img_file, (offset_x, offset_y) in offsets.items():
    img_path = os.path.join(images_folder, img_file)
    small_output_path = os.path.join(small_output_folder, img_file)
    annotation_path = os.path.join(annotations_folder, f"{os.path.splitext(img_file)[0]}.json")

    # 確認文件存在
    if not os.path.exists(img_path):
        print(f"Warning: {img_file} not found in {images_folder}, skipping...")
        continue
    if not os.path.exists(small_output_path):
        print(f"Warning: {img_file} not found in {small_output_folder}, skipping...")
        continue
    if not os.path.exists(annotation_path):
        print(f"Warning: Annotation for {img_file} not found, skipping...")
        continue

    # 加載圖片
    image = cv2.imread(img_path)

    # 讀取 Ground Truth BBox
    with open(annotation_path, "r") as f:
        gt_data = json.load(f)

    # 檢查 JSON 結構
    if isinstance(gt_data, list):
        annotations = gt_data
    elif isinstance(gt_data, dict) and "annotations" in gt_data:
        annotations = gt_data["annotations"]
    else:
        print(f"Invalid JSON format in {annotation_path}, skipping...")
        continue

    gt_bboxes = []
    for obj in annotations:
        if obj.get("name") is None or obj.get("bbox") is None:
            continue
        # bbox 是四個頂點的座標
        bbox = obj["bbox"]
        if len(bbox) != 4:
            print(f"Invalid bbox format for object {obj}, skipping...")
            continue

        # 存入 Ground Truth 四邊形頂點
        gt_bboxes.append(np.array(bbox, dtype=float))

    # 推理模型
    results = model(small_output_path)
    for result in results:
        detections = sv.Detections.from_ultralytics(result)

        # 如果沒有四點座標，跳過
        if "xyxyxyxy" not in detections.data:
            print(f"No bounding box data for {img_file}, skipping...")
            continue

        # 偏移預測框，並畫在圖片上
        for bbox in detections.data["xyxyxyxy"]:
            # 構建偏移後的四邊形頂點
            bbox = np.array(bbox, dtype=float).flatten()
            bbox_offset = np.array(bbox, dtype=float) + np.tile([offset_x, offset_y], 4)
            polygon_points = bbox_offset.reshape((-1, 2))
            cv2.drawContours(image, [np.int0(polygon_points)], -1, (255, 0, 0), 2)  # 藍色框（預測）

        # 計算 IoU
        for pred_bbox in detections.data["xyxyxyxy"]:
            pred_bbox = np.array(pred_bbox, dtype=float).flatten()
            pred_polygon = np.array(pred_bbox, dtype=float) + np.tile([offset_x, offset_y], 4)
            pred_polygon = pred_polygon.reshape((-1, 2))
            iou_scores = [calculate_polygon_iou(pred_polygon, gt_bbox) for gt_bbox in gt_bboxes]
            if iou_scores:
                mean_iou_list.append(max(iou_scores))

    # 繪製 Ground Truth 框
    for gt_box in gt_bboxes:
        cv2.drawContours(image, [np.int0(gt_box)], -1, (0, 0, 255), 2)  # 紅色框（Ground Truth）

    # 儲存繪製後的圖片
    output_path = os.path.join(completed_output_folder, img_file)
    cv2.imwrite(output_path, image)

# 計算並打印 Mean IoU
mean_iou = sum(mean_iou_list) / len(mean_iou_list) if mean_iou_list else 0
print(f"Mean IoU: {mean_iou}")


image 1/1 d:\Azure\\\Image_processing\UI\temp\output_1\cropped\00075616-AP0.jpg: 416x384 689.0ms
Speed: 2.0ms preprocess, 689.0ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 384)



image 1/1 d:\Azure\\\Image_processing\UI\temp\output_1\cropped\00075616-LA0.jpg: 320x416 498.0ms
Speed: 3.0ms preprocess, 498.0ms inference, 3.0ms postprocess per image at shape (1, 3, 320, 416)



image 1/1 d:\Azure\\\Image_processing\UI\temp\output_1\cropped\00169382 L 51F AP0.jpg: 416x416 (no detections), 652.0ms
Speed: 2.0ms preprocess, 652.0ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 d:\Azure\\\Image_processing\UI\temp\output_1\cropped\00169382 L 51F LAT0.jpg: 288x416 419.0ms
Speed: 1.0ms preprocess, 419.0ms inference, 2.0ms postprocess per image at shape (1, 3, 288, 416)



image 1/1 d:\Azure\\\Image_processing\UI\temp\output_1\cropped\00169382 R 51F AP0.jpg: 416x416 (no detections), 570.0ms
Speed: 2.0ms preprocess, 570.0ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 d:\Azure\\\Image_processing\UI\temp\output_1\cropped\00169382 R 51F LAT0.jpg: 288x416 439.0ms
Speed: 1.0ms preprocess, 439.0ms inference, 1.0ms postprocess per image at shape (1, 3, 288, 416)



image 1/1 d:\Azure\\\Image_processing\UI\temp\output_1\cropped\00230304 R 50F AP0.jpg: 416x384 (no detections), 628.1ms
Speed: 1.0ms preprocess, 628.1ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 384)

image 1/1 d:\Azure\\\Image_processing\UI\temp\output_1\cropped\00230304 R 50F LAT0.jpg: 352x416 (no detections), 504.0ms
Speed: 1.0ms preprocess, 504.0ms inference, 0.0ms postprocess per image at shape (1, 3, 352, 416)

image 1/1 d:\Azure\\\Image_processing\UI\temp\output_1\cropped\00454212-RAP0.jpg: 416x416 571.0ms
Speed: 1.0ms preprocess, 571.0ms inference, 2.0ms postprocess per image at shape (1, 3, 416, 416)



image 1/1 d:\Azure\\\Image_processing\UI\temp\output_1\cropped\00454212-RLA0.jpg: 288x416 476.9ms
Speed: 2.0ms preprocess, 476.9ms inference, 3.6ms postprocess per image at shape (1, 3, 288, 416)
Mean IoU: 0.8101910226119272


# 產生test data （將annotations座標合併） 產生完整圖片的fracture座標

In [9]:
import os
import json

# 文件夹路径
images_folder = "./data/sampled_data/images"
annotations_folder = "./data/sampled_data/annotations"
annotations_fracture_folder = "./data/sampled_data/fracture_annotations"
annotations_output_folder = "./data/annotations_output"

# 创建输出文件夹
os.makedirs(annotations_output_folder, exist_ok=True)

# 读取所有 annotations 和 annotations_fracture 文件
annotation_files = [f for f in os.listdir(annotations_folder) if f.endswith('.json')]
fracture_files = [f for f in os.listdir(annotations_fracture_folder) if f.endswith('.json')]

# 遍历每个 annotation 文件
for annotation_file in annotation_files:
    # 获取对应的图像文件名
    image_file = annotation_file.replace('.json', '.jpg')
    image_path = os.path.join(images_folder, image_file)
    if not os.path.exists(image_path):
        print(f"Warning: Image {image_file} not found in {images_folder}, skipping...")
        continue

    # 读取对应的 annotations 和 annotations_fracture 文件
    annotation_path = os.path.join(annotations_folder, annotation_file)
    fracture_path = os.path.join(annotations_fracture_folder, annotation_file)

    if not os.path.exists(fracture_path):
        print(f"Warning: Fracture data {annotation_file} not found, skipping...")
        continue

    with open(annotation_path, 'r') as f:
        annotation_data = json.load(f)
    with open(fracture_path, 'r') as f:
        fracture_data = json.load(f)

    # 获取 annotations 的 bbox（左上角和右下角）
    if len(annotation_data) == 0 or "bbox" not in annotation_data[0]:
        print(f"Warning: No bbox in {annotation_file}, writing null annotation...")
        output_data = [{"name": None, "bbox": None}]
    else:
        bbox = annotation_data[0]["bbox"]
        x1, y1 = int(bbox[0]), int(bbox[1])

        # 获取 annotations_fracture 的四点坐标
        if len(fracture_data) == 0 or fracture_data[0]["name"] is None:
            print(f"No Fracture in {annotation_file}, writing null annotation...")
            output_data = [{"name": None, "bbox": None}]
        else:
            fracture_bbox = fracture_data[0]["bbox"]
            if fracture_bbox is None:
                print(f"No bbox data in fracture file {annotation_file}, writing null annotation...")
                output_data = [{"name": None, "bbox": None}]
            else:
                # 偏移 fracture_bbox 坐标
                offset_bbox = []
                for point in fracture_bbox:
                    offset_bbox.append([point[0] + x1, point[1] + y1])
                output_data = [{"name": "Fracture", "bbox": offset_bbox}]

    # 保存到输出文件夹
    output_path = os.path.join(annotations_output_folder, annotation_file)
    with open(output_path, 'w') as f:
        json.dump(output_data, f, indent=4)
    print(f"Annotation file saved to {output_path}")

print("All annotations processed and saved in ./final/annotations_output.")

Annotation file saved to ./data/annotations_output\00075616-AP0.json
No Fracture in 00169382 L 51F AP0.json, writing null annotation...
Annotation file saved to ./data/annotations_output\00169382 L 51F AP0.json
Annotation file saved to ./data/annotations_output\00169382 L 51F LAT0.json
Annotation file saved to ./data/annotations_output\00230304 R 50F LAT0.json
No Fracture in 01299357 R 80M LAT0.json, writing null annotation...
Annotation file saved to ./data/annotations_output\01299357 R 80M LAT0.json
No Fracture in 04718312 L 53 F AP0.json, writing null annotation...
Annotation file saved to ./data/annotations_output\04718312 L 53 F AP0.json
No Fracture in 05154334 L 77 F AP0.json, writing null annotation...
Annotation file saved to ./data/annotations_output\05154334 L 77 F AP0.json
No Fracture in 06117033 R 61F AP0.json, writing null annotation...
Annotation file saved to ./data/annotations_output\06117033 R 61F AP0.json
Annotation file saved to ./data/annotations_output\06598570-LAT

In [2]:
import os
import json
import shutil

# 文件夹路径
images_folder = "./temp/data/scaphoid_detection/images"
annotations_folder = "./temp/data/scaphoid_detection/annotations"
annotations_fracture_folder = "./temp/data/fracture_detection/annotations"
output_data_folder = "./temp/data"

# 创建輸出資料夾
output_annotations_folder = os.path.join(output_data_folder, "annotations")
output_images_folder = os.path.join(output_data_folder, "images")
output_scaphoid_annotations_folder = os.path.join(output_data_folder, "scaphoid_annotations")

os.makedirs(output_annotations_folder, exist_ok=True)
os.makedirs(output_images_folder, exist_ok=True)
os.makedirs(output_scaphoid_annotations_folder, exist_ok=True)

# 读取所有 annotations 和 annotations_fracture 文件
annotation_files = [f for f in os.listdir(annotations_folder) if f.endswith('.json')]
fracture_files = [f for f in os.listdir(annotations_fracture_folder) if f.endswith('.json')]

# 遍历每个 annotation 文件
for annotation_file in annotation_files:
    # 获取对应的图像文件名
    image_file = annotation_file.replace('.json', '.jpg')
    image_path = os.path.join(images_folder, image_file)
    if not os.path.exists(image_path):
        print(f"Warning: Image {image_file} not found in {images_folder}, skipping...")
        continue

    # 读取对应的 annotations 和 annotations_fracture 文件
    annotation_path = os.path.join(annotations_folder, annotation_file)
    fracture_path = os.path.join(annotations_fracture_folder, annotation_file)

    if not os.path.exists(fracture_path):
        print(f"Warning: Fracture data {annotation_file} not found, skipping...")
        continue

    with open(annotation_path, 'r') as f:
        annotation_data = json.load(f)
    with open(fracture_path, 'r') as f:
        fracture_data = json.load(f)

    # 获取 annotations 的 bbox（左上角和右下角）
    if len(annotation_data) == 0 or "bbox" not in annotation_data[0]:
        print(f"Warning: No bbox in {annotation_file}, writing null annotation...")
        output_data = [{"name": None, "bbox": None}]
    else:
        bbox = annotation_data[0]["bbox"]
        x1, y1 = int(bbox[0]), int(bbox[1])

        # 获取 annotations_fracture 的四点坐标
        if len(fracture_data) == 0 or fracture_data[0]["name"] is None:
            print(f"No Fracture in {annotation_file}, writing null annotation...")
            output_data = [{"name": None, "bbox": None}]
        else:
            fracture_bbox = fracture_data[0]["bbox"]
            if fracture_bbox is None:
                print(f"No bbox data in fracture file {annotation_file}, writing null annotation...")
                output_data = [{"name": None, "bbox": None}]
            else:
                # 偏移 fracture_bbox 坐标
                offset_bbox = []
                for point in fracture_bbox:
                    offset_bbox.append([point[0] + x1, point[1] + y1])
                output_data = [{"name": "Fracture", "bbox": offset_bbox}]

    # 保存到输出文件夹
    output_path = os.path.join(output_annotations_folder, annotation_file)
    with open(output_path, 'w') as f:
        json.dump(output_data, f, indent=4)
    print(f"Annotation file saved to {output_path}")

# 複製原始圖像文件夾到輸出資料夾
shutil.copytree(images_folder, output_images_folder, dirs_exist_ok=True)

# 複製 scaphoid_annotations 文件夹到輸出資料夾
shutil.copytree(annotations_folder, output_scaphoid_annotations_folder, dirs_exist_ok=True)

# 刪除多餘資料夾，只保留指定的三個資料夾
for item in os.listdir(output_data_folder):
    item_path = os.path.join(output_data_folder, item)
    if item not in ["annotations", "images", "scaphoid_annotations"]:
        if os.path.isdir(item_path):
            shutil.rmtree(item_path)
        else:
            os.remove(item_path)

print("All annotations processed, saved, and unnecessary folders removed.")

Annotation file saved to ./temp/data\annotations\00075616-AP0.json
Annotation file saved to ./temp/data\annotations\00075616-LA0.json
No Fracture in 00169382 L 51F AP0.json, writing null annotation...
Annotation file saved to ./temp/data\annotations\00169382 L 51F AP0.json
Annotation file saved to ./temp/data\annotations\00169382 L 51F LAT0.json
No Fracture in 00169382 R 51F AP0.json, writing null annotation...
Annotation file saved to ./temp/data\annotations\00169382 R 51F AP0.json
Annotation file saved to ./temp/data\annotations\00169382 R 51F LAT0.json
No Fracture in 00230304 R 50F AP0.json, writing null annotation...
Annotation file saved to ./temp/data\annotations\00230304 R 50F AP0.json
Annotation file saved to ./temp/data\annotations\00230304 R 50F LAT0.json
Annotation file saved to ./temp/data\annotations\00454212-RAP0.json
Annotation file saved to ./temp/data\annotations\00454212-RLA0.json
No Fracture in 00952402 R 70F AP0.json, writing null annotation...
Annotation file saved

# Sample 20

In [6]:
import os
import random
import shutil

# 定義資料夾路徑
data_dir = "data"
images_dir = os.path.join(data_dir, "images")
annotations_dir = os.path.join(data_dir, "annotations")
fracture_annotations_dir = os.path.join(data_dir, "fracture_annotations")

# 確認資料夾存在
if not all(os.path.exists(d) for d in [images_dir, annotations_dir, fracture_annotations_dir]):
    raise FileNotFoundError("其中一個或多個資料夾不存在，請確認資料夾路徑。")

# 取得所有檔案名稱（不含副檔名）的交集
images_files = {os.path.splitext(f)[0] for f in os.listdir(images_dir) if f.endswith('.jpg')}
annotations_files = {os.path.splitext(f)[0] for f in os.listdir(annotations_dir) if f.endswith('.json')}
fracture_annotations_files = {os.path.splitext(f)[0] for f in os.listdir(fracture_annotations_dir) if f.endswith('.json')}

common_files = images_files & annotations_files & fracture_annotations_files

# 確保有足夠的檔案來取樣
if len(common_files) < 20:
    raise ValueError("資料夾中共同的檔案數量不足20個。")

# 隨機取樣20個檔案
sampled_files = random.sample(list(common_files), 20)  # 將集合轉為列表

# 定義輸出資料夾（可選）
output_dir = "./data/sampled_data"
os.makedirs(output_dir, exist_ok=True)
os.makedirs(os.path.join(output_dir, "images"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "annotations"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "fracture_annotations"), exist_ok=True)

# 複製取樣的檔案到輸出資料夾
for file_name in sampled_files:
    shutil.copy(os.path.join(images_dir, f"{file_name}.jpg"), os.path.join(output_dir, "images", f"{file_name}.jpg"))
    shutil.copy(os.path.join(annotations_dir, f"{file_name}.json"), os.path.join(output_dir, "annotations", f"{file_name}.json"))
    shutil.copy(os.path.join(fracture_annotations_dir, f"{file_name}.json"), os.path.join(output_dir, "fracture_annotations", f"{file_name}.json"))

print(f"已成功取樣並複製 20 個檔案到資料夾：{output_dir}")


已成功取樣並複製 20 個檔案到資料夾：sampled_data
